In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\Repos\\BrainTumorClassfication-DL'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_include_top: bool
    params_epochs: int
    params_device: str


In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import  read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    def get_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_basemodel
        params = self.params
        create_directories([config.root_dir])

        base_model_config = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path=config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size= params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_classes=params.CLASSES,
            params_include_top = params.INCLUDE_TOP,
            params_epochs=params.EPOCHS,
            params_device = params.DEVICE
        )
        return base_model_config
    


In [6]:
import os
import urllib.request as request
import zipfile
from cnnClassifier.logging import logger
from cnnClassifier.utils.common import get_size
import torchvision.models as models
import torch
from torch import nn
import torchinfo
from torchinfo import summary

torch.manual_seed(42)
torch.cuda.manual_seed(42)


In [7]:
# try:
#   import torchinfo
# except:
#   !pip install torchinfo
#   import torchinfo

# from torchinfo import summary

In [10]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config =config
    
    def get_base_model(self):
        try:
            self.model = models.resnet18(pretrained=True)
            torch.save(self.model,self.config.base_model_path)
        except Exception  as e:
            raise e

    @staticmethod
    def _prepare_full_model(model,classes:int,device:str):
        model.fc = nn.Linear(model.fc.in_features, classes)
        model = model.to(device)
        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model= self.model,
            classes=self.config.params_classes,
            device=self.config.params_device
        )
        logger.info(f"Model: \n {summary(self.full_model, input_size=[1,3, 224,224])}")

        torch.save(self.full_model,self.config.updated_base_model_path)

In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e


[2025-04-22 09:11:28,339: INFO: common: yaml file: config\config.yaml loaded succesfully..]
[2025-04-22 09:11:28,346: INFO: common: yaml file: params.yaml loaded succesfully..]
[2025-04-22 09:11:28,349: INFO: common: Create Directory at :artifacts]
[2025-04-22 09:11:28,352: INFO: common: Create Directory at :artifacts/perpare_basemodel]
[2025-04-22 09:11:29,262: INFO: 1409500857: Model: 
Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 4]                    --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                 

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\sri\.conda\envs\cv-env\lib\logging\__init__.py", line 1103, in emit
    stream.write(msg + self.terminator)
  File "c:\Users\sri\.conda\envs\cv-env\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 385-386: character maps to <undefined>
Call stack:
  File "c:\Users\sri\.conda\envs\cv-env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\sri\.conda\envs\cv-env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\sri\.conda\envs\cv-env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sri\.conda\envs\cv-env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sri\.conda\e